In [1]:
import os
import torch
import time
from collections import defaultdict
import numpy as np

from models.networks import NGP
from models.rendering import render

from datasets import dataset_dict
from utils import load_ckpt
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from datasets import dataset_dict


# Load model and data

In [ ]:
# Change here #
# img_wh = (800, 800) # full resolution of the input images
# dataset_name = 'llff' # blender or llff (own data)
# scene_name = 'test2' # whatever you want
# root_dir = 'test2/' # the folder containing data
# ckpt_path = 'ckpts/exp2/epoch=29.ckpt' # the model path
###############
# dataset_name = 'colmap'
# scene = 'test'
# dataset = dataset_dict[dataset_name](
#     f'/data2_12t/dataset/3D_Project',
#     split='test', downsample=1.0/2
# )

dataset_name = 'nerf'
scene = 'apple'
dataset = dataset_dict[dataset_name](
    f'/data2_12t/dataset/OpenXD-OmniObject3D-New/raw/blender_renders/apple_001/render/',
    split='test', downsample=1.0/4
)

# dataset_name = 'nsvf'
# scene = 'Barn'
# dataset = dataset_dict[dataset_name](
#     f'/data2_12t/dataset/TanksAndTemple/{scene}',
#     split='test', downsample=1.0/4
# )


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
model = NGP(scale=0.5).cuda()
load_ckpt(model, f'ckpts/{dataset_name}/{scene}/epoch=29_slim.ckpt')

In [ ]:
from kornia.utils.grid import create_meshgrid3d
import vren

xyz = create_meshgrid3d(model.grid_size, model.grid_size, model.grid_size, False, dtype=torch.int32).reshape(-1, 3)
_density_bitfield = model.density_bitfield
density_bitfield = torch.zeros(model.cascades*model.grid_size**3//8, 8, dtype=torch.bool)
for i in range(8):
    density_bitfield[:, i] = _density_bitfield & torch.tensor([2**i], device='cuda')
density_bitfield = density_bitfield.reshape(model.cascades, model.grid_size**3).cpu()
indices = vren.morton3D(xyz.cuda()).long()

# Search for tight bounds of the object (trial and error!)

In [ ]:
# ### Tune these parameters until the whole object lies tightly in range with little noise ###

import mcubes
import trimesh

N = 400 # controls the resolution, set this number small here because we're only finding
        # good ranges here, not yet for mesh reconstruction; we can set this number high
        # when it comes to final reconstruction.
xmin, xmax = -0.5, 0.5 # left/right range
ymin, ymax = -0.5, 0.5 # forward/backward range
zmin, zmax = -0.5, 0.5 # up/down range
## Attention! the ranges MUST have the same length!
sigma_threshold = 20. # controls the noise (lower=maybe more noise; higher=some mesh might be missing)
############################################################################################

x = np.linspace(xmin, xmax, N)
y = np.linspace(ymin, ymax, N)
z = np.linspace(zmin, zmax, N)

xyz_ = torch.FloatTensor(np.stack(np.meshgrid(x, y, z), -1).reshape(-1, 3)).cuda()

with torch.no_grad():
    sigma = model.density(xyz_).cpu().numpy().astype(np.float32)
    
sigma = np.maximum(sigma, 2.0)
sigma = sigma.reshape(N, N, N)
# The below lines are for visualization, COMMENT OUT once you find the best range and increase N!
vertices, triangles = mcubes.marching_cubes(sigma, sigma_threshold)
mesh = trimesh.Trimesh(vertices/N, triangles)
mesh.show()

# Extract colored mesh

Once you find the best range, now **RESTART** the notebook, and copy the configs to the following cell
and execute it.

In [9]:
# Copy the variables you have above here! ####
img_wh = (800, 800) # full resolution of the input images
dataset_name = 'nerf' # blender or llff (own data)
scene = 'apple' # whatever you want
root_dir = f'/data2_12t/dataset/OpenXD-OmniObject3D-New/raw/blender_renders/{scene}_001/render/' # the folder containing data
ckpt_path = f'ckpts/{dataset_name}/{scene}/epoch=29_slim.ckpt' # the model path

N = 400        
xmin, xmax = -0.5, 0.5 # left/right range
ymin, ymax = -0.5, 0.5 # forward/backward range
zmin, zmax = -0.5, 0.5 # up/down range
## Attention! the ranges MUST have the same length!
sigma_threshold = 20. # controls the noise (lower=maybe more noise; higher=some mesh might be missing)
###############################################

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['ROOT_DIR'] = root_dir
os.environ['DATASET_NAME'] = dataset_name
os.environ['SCENE'] = scene
os.environ['IMG_SIZE'] = f"{img_wh[0]} {img_wh[1]}"
os.environ['CKPT_PATH'] = ckpt_path
os.environ['N_GRID'] = f"{N}" # final resolution. You can set this number high to preserve more details
os.environ['X_RANGE'] = f"{xmin} {xmax}"
os.environ['Y_RANGE'] = f"{ymin} {ymax}"
os.environ['Z_RANGE'] = f"{zmin} {zmax}"
os.environ['SIGMA_THRESHOLD'] = str(sigma_threshold)
os.environ['OCC_THRESHOLD'] = "0.2" # probably doesn't require tuning. If you find the color is not close
                                    # to real, try to set this number smaller (the effect of this number
                                    # is explained in my youtube video)

!python extract_color_mesh.py \
    --root_dir $ROOT_DIR \
    --dataset_name $DATASET_NAME \
    --scene $SCENE \
    --img_wh $IMG_SIZE \
    --ckpt_path $CKPT_PATH \
    --N_grid $N_GRID \
    --x_range $X_RANGE \
    --y_range $Y_RANGE \
    --z_range $Z_RANGE \
    --sigma_threshold $SIGMA_THRESHOLD \
    --occ_threshold $OCC_THRESHOLD 

/data2_12t/user/wlz/anaconda3/envs/ngp/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Loading 100 test images ...
100%|█████████████████████████████████████████| 100/100 [00:04<00:00, 23.96it/s]
/data2_12t/user/wlz/Code/ngp_pl-master/datasets/nerf.py:93: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  self.poses = torch.FloatTensor(self.poses) # (N_images, 3, 4)
GridEncoding: Nmin=16 b=1.31951 F=2 T=2^19 L=16
Predicting occupancy ...
Extra